In [23]:
import os, sys

base = "/kaggle/input/datasets"
print("Level 1:", os.listdir(base))

found = None

for user_folder in os.listdir(base):
    user_path = f"{base}/{user_folder}"
    if not os.path.isdir(user_path):
        continue

    # List datasets under your username folder (kgaero)
    for ds in os.listdir(user_path):
        ds_path = f"{user_path}/{ds}"
        if not os.path.isdir(ds_path):
            continue

        # Look for the project folder
        if "language-translation" in os.listdir(ds_path):
            found = f"{ds_path}/language-translation"
            print("FOUND:", found)
            print("Files:", os.listdir(found))
            break

    if found:
        break

assert found, "Did not find language-translation folder anywhere under /kaggle/input/datasets"
sys.path.insert(0, found)

import config
print("Imported config from:", config.__file__)

Level 1: ['kgaero']
FOUND: /kaggle/input/datasets/kgaero/trained-language-translation/language-translation
Files: ['runs', 'train.py', 'KAGGLE.md', 'model.py', 'tokenizer_en.json', 'tokenizer_it.json', 'config.py', 'opus_books_weights', 'requirements.txt', 'run_kaggle.sh', 'dataset.py', '.git']
Imported config from: /kaggle/input/datasets/kgaero/trained-language-translation/language-translation/config.py


In [26]:
PROJECT_DIR = found  # this is .../language-translation
WEIGHTS_DIR = f"{PROJECT_DIR}/opus_books_weights"
print("Weights dir:", WEIGHTS_DIR)
print("Weights files:", sorted([f for f in os.listdir(WEIGHTS_DIR) if f.endswith(".pt")])[:10])

Weights dir: /kaggle/input/datasets/kgaero/trained-language-translation/language-translation/opus_books_weights
Weights files: ['tmodel_00.pt', 'tmodel_01.pt', 'tmodel_02.pt', 'tmodel_03.pt', 'tmodel_04.pt', 'tmodel_05.pt', 'tmodel_06.pt', 'tmodel_07.pt', 'tmodel_08.pt', 'tmodel_09.pt']


In [24]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation

2026-02-21 05:22:26.308048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771651346.737248      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771651346.850398      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771651347.848409      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771651347.848449      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771651347.848451      55 computation_placer.cc:177] computation placer alr

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# model_filename = get_weights_file_path(config, f"19")
# state= torch.load(model_filename)
# model.load_state_dict(state['model_state_dict'])


using device: cuda
Filtered dataset size: 32332
Max length of source sentence: 309
Max length of target sentence: 274


In [31]:
ckpts = sorted([f for f in os.listdir(WEIGHTS_DIR) if f.endswith(".pt")])
print("All checkpoints:", ckpts)

model_filename = os.path.join(WEIGHTS_DIR, ckpts[-1])
print("Auto-loading:", model_filename)

state = torch.load(model_filename, map_location=device)
model.load_state_dict(state["model_state_dict"])

All checkpoints: ['tmodel_00.pt', 'tmodel_01.pt', 'tmodel_02.pt', 'tmodel_03.pt', 'tmodel_04.pt', 'tmodel_05.pt', 'tmodel_06.pt', 'tmodel_07.pt', 'tmodel_08.pt', 'tmodel_09.pt', 'tmodel_10.pt', 'tmodel_11.pt', 'tmodel_12.pt', 'tmodel_13.pt', 'tmodel_14.pt', 'tmodel_15.pt', 'tmodel_16.pt', 'tmodel_17.pt', 'tmodel_18.pt', 'tmodel_19.pt']
Auto-loading: /kaggle/input/datasets/kgaero/trained-language-translation/language-translation/opus_books_weights/tmodel_19.pt


<All keys matched successfully>

In [34]:
run_validation(
    model=model,
    validation_ds=val_dataloader,     # or val_dataloader depending on your code
    tokenizer_tgt=tokenizer_tgt,
    max_len=config["seq_len"],
    device=device,
    print_msg=lambda msg: print(msg),
    global_step=0,
    writer=None,
    num_examples=10,
)

stty: 'standard input': Inappropriate ioctl for device


--------------------------------------------------------------------------------
    SOURCE: 'And was he very glad?' he asked.
    TARGET: — Ebbene, era contento? — chiese.
 PREDICTED: — E fu molto contento ? — chiese .
--------------------------------------------------------------------------------
    SOURCE: I pray God Mrs. Fairfax may not turn out a second Mrs. Reed; but if she does, I am not bound to stay with her! let the worst come to the worst, I can advertise again. How far are we on our road now, I wonder?"
    TARGET: In ogni modo, non sono obbligata a rimanere da lei; potrei trovarmi un altro posto.
 PREDICTED: In ogni modo , non è obbligata a rimanere da lei ; potrei trovarmi un altro posto .
--------------------------------------------------------------------------------
    SOURCE: Now he was pleased and excited by the visit to his studio of these grand Russians who had come in a carriage.
    TARGET: Ora lo rallegrava e lo agitava la visita del suo studio da parte di qu